# Reading Databases and Data Sources

This notebook covers querying databases and data sources in Notion:
- Understanding databases vs data sources
- Retrieving database metadata
- Querying pages from data sources
- Filtering and sorting query results
- Working with query pagination

## Prerequisites

Make sure you have set these environment variables:
- `NOTION_API_TOKEN` - Your Notion integration token
- `NOTION_TEST_PAGE_ID` - A test page ID where we can create test databases

## Setup: Load Dependencies and Initialize Client

In [1]:
// Load the Kotlin Notion Client library from Maven Central
@file:DependsOn("it.saabel:kotlin-notion-client:0.1.0")

// Import necessary classes
import it.saabel.kotlinnotionclient.NotionClient
import it.saabel.kotlinnotionclient.models.base.SelectOptionColor
import it.saabel.kotlinnotionclient.models.datasources.SortDirection
import it.saabel.kotlinnotionclient.models.pages.PageProperty
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.async
import kotlinx.coroutines.awaitAll

// Initialize the client
val apiToken = System.getenv("NOTION_API_TOKEN")
    ?: error("❌ NOTION_API_TOKEN environment variable not set")

val parentPageId = System.getenv("NOTION_TEST_PAGE_ID")
    ?: error("❌ NOTION_TEST_PAGE_ID environment variable not set")

val notion = NotionClient(apiToken)

println("✅ NotionClient initialized successfully!")

✅ NotionClient initialized successfully!


## Understanding Databases vs Data Sources

In Notion:
- A **Database** is a container that can hold multiple tables/views
- A **Data Source** is a specific table within a database that stores the actual data
- When you create a database, a default data source is automatically created
- To query pages, you need the data source ID, not the database ID

## Example 1: Create a Test Database

Let's create a simple task database for our examples.

In [2]:
val database = runBlocking {
    notion.databases.create {
        parent.page(parentPageId)
        title("Task Tracker")

        properties {
            title("Task Name")
            select("Status") {
                option("To Do", SelectOptionColor.RED)
                option("In Progress", SelectOptionColor.YELLOW)
                option("Done", SelectOptionColor.GREEN)
            }
            number("Priority")
            date("Due Date")
            checkbox("Completed")
        }
    }
}

// Get the data source ID - we'll need this to query pages
val dataSourceId = database.dataSources.firstOrNull()?.id
    ?: error("Database has no data sources")

println("✅ Database created successfully!")
println("   Database ID: ${database.id}")
println("   Data Source ID: $dataSourceId")
println("   Title: ${database.title.firstOrNull()?.plainText}")
println("   URL: ${database.url}")

✅ Database created successfully!
   Database ID: 82cbe8f6-8ad1-4b48-92f4-30cd13de9dba
   Data Source ID: f40c7689-f1b5-43b0-aa24-cbc992b8492d
   Title: Task Tracker
   URL: https://www.notion.so/82cbe8f68ad14b4892f430cd13de9dba


## Example 2: Add Some Test Data

Let's populate the database with some sample tasks. We'll use Kotlin coroutines to create all pages concurrently for better performance.

In [3]:
// Create a few test pages
val taskData = listOf(
    Triple("Write documentation", "In Progress", 8.0),
    Triple("Fix bug in authentication", "To Do", 9.0),
    Triple("Implement new feature", "To Do", 5.0),
    Triple("Update dependencies", "Done", 3.0),
    Triple("Review pull requests", "In Progress", 7.0)
)

// Use coroutines to create all pages concurrently for better performance
runBlocking {
    taskData.map { (taskName, status, priority) ->
        async {
            notion.pages.create {
                parent.dataSource(dataSourceId)
                properties {
                    title("Task Name", taskName)
                    select("Status", status)
                    number("Priority", priority)
                }
            }
        }
    }.awaitAll()
}

println("✅ Created ${taskData.size} test tasks concurrently")

✅ Created 5 test tasks concurrently


## Example 4: Retrieve Data Source Schema

Now let's look at the data source to see its schema (property definitions).

In [4]:
val retrievedDb = runBlocking {
    notion.databases.retrieve(database.id)
}

println("📊 Database Information:")
println("   Title: ${retrievedDb.title.firstOrNull()?.plainText}")
println("   Created: ${retrievedDb.createdTime}")
println("   Last edited: ${retrievedDb.lastEditedTime}")
println("   Archived: ${retrievedDb.archived}")
println("   In trash: ${retrievedDb.inTrash}")

println("\n📋 Data Sources:")
retrievedDb.dataSources.forEach { println("   - ID: ${it.id}")
}

📊 Database Information:
   Title: Task Tracker
   Created: 2025-10-17T07:04:40.316+00:00
   Last edited: 2025-10-17T07:04:40.316+00:00
   Archived: false
   In trash: false

📋 Data Sources:
   - ID: f40c7689-f1b5-43b0-aa24-cbc992b8492d


In [5]:
val dataSource = runBlocking {
    notion.dataSources.retrieve(dataSourceId)
}

println("📋 Data Source Schema:")
println("   Name: ${dataSource.title.firstOrNull()?.plainText}")
println("   Properties:")

dataSource.properties.forEach { (propName, config) ->
    println("      - $propName: ${config.type}")
}

📋 Data Source Schema:
   Name: Task Tracker
   Properties:
      - Priority: number
      - Due Date: date
      - Status: select
      - Completed: checkbox
      - Task Name: title


## Example 5: Query All Pages (Simple Query)

The simplest way to get pages from a data source.

In [6]:
val allPages = runBlocking {
    notion.dataSources.query(dataSourceId) {}
}

println("📄 All Pages (${allPages.size} total):")
allPages.forEach { page ->
    val titleProp = page.properties["Task Name"] as? PageProperty.Title
    val title = titleProp?.plainText ?: "Untitled"
    
    val statusProp = page.properties["Status"] as? PageProperty.Select
    val status = statusProp?.select?.name ?: "No status"
    
    println("   - $title [$status]")
}

📄 All Pages (5 total):
   - Review pull requests [In Progress]
   - Update dependencies [Done]
   - Implement new feature [To Do]
   - Fix bug in authentication [To Do]
   - Write documentation [In Progress]


## Example 6: Query with Filters

Let's filter to show only tasks with status "To Do".

In [7]:
val todoPages = runBlocking {
    notion.dataSources.query(dataSourceId) {
        filter {
            select("Status").equals("To Do")
        }
    }
}

println("📋 To Do Tasks (${todoPages.size} total):")
todoPages.forEach { page ->
    val titleProp = page.properties["Task Name"] as? PageProperty.Title
    val title = titleProp?.plainText ?: "Untitled"
    
    val priorityProp = page.properties["Priority"] as? PageProperty.Number
    val priority = priorityProp?.number ?: 0.0
    
    println("   - $title (Priority: $priority)")
}

📋 To Do Tasks (2 total):
   - Implement new feature (Priority: 5.0)
   - Fix bug in authentication (Priority: 9.0)


## Example 7: Query with Complex Filters

Combine multiple conditions using `and()` and `or()`.

In [8]:
val urgentTasks = runBlocking {
    notion.dataSources.query(dataSourceId) {
        filter {
            and(
                select("Status").equals("To Do"),
                number("Priority").greaterThan(7.0)
            )
        }
    }
}

println("🔥 Urgent To Do Tasks (${urgentTasks.size} total):")
urgentTasks.forEach { page ->
    val titleProp = page.properties["Task Name"] as? PageProperty.Title
    val title = titleProp?.plainText ?: "Untitled"
    
    val priorityProp = page.properties["Priority"] as? PageProperty.Number
    val priority = priorityProp?.number ?: 0.0
    
    println("   - $title (Priority: $priority)")
}

🔥 Urgent To Do Tasks (1 total):
   - Fix bug in authentication (Priority: 9.0)


## Example 8: Query with Sorting

Sort results by priority in descending order.

In [9]:
val sortedPages = runBlocking {
    notion.dataSources.query(dataSourceId) {
        sortBy("Priority", SortDirection.DESCENDING)
    }
}

println("📊 Tasks by Priority (highest first):")
sortedPages.forEach { page ->
    val titleProp = page.properties["Task Name"] as? PageProperty.Title
    val title = titleProp?.plainText ?: "Untitled"

    val priorityProp = page.properties["Priority"] as? PageProperty.Number
    val priority = priorityProp?.number ?: 0.0

    val statusProp = page.properties["Status"] as? PageProperty.Select
    val status = statusProp?.select?.name ?: "No status"

    println("   - [Priority $priority] $title - $status")
}

📊 Tasks by Priority (highest first):
   - [Priority 9.0] Fix bug in authentication - To Do
   - [Priority 8.0] Write documentation - In Progress
   - [Priority 7.0] Review pull requests - In Progress
   - [Priority 5.0] Implement new feature - To Do
   - [Priority 3.0] Update dependencies - Done


## Cleanup

Archive the test database when done.

In [11]:
// Archive the test database to clean up
runBlocking {
    notion.databases.archive(database.id)
}

println("✅ Test database archived successfully!")

✅ Test database archived successfully!


## Next Steps

Now that you understand querying databases and data sources, explore:
- **[03-creating-pages.ipynb](./03-creating-pages.ipynb)** - Create pages with properties
- **[04-working-with-blocks.ipynb](./04-working-with-blocks.ipynb)** - Build page content with blocks
- **[05-rich-text-dsl.ipynb](./05-rich-text-dsl.ipynb)** - Format text with the Rich Text DSL

## Key Takeaways

- **Database vs Data Source**: Databases are containers; data sources are the actual tables with data
- **Getting Data Source ID**: Access via `database.dataSources.firstOrNull()?.id`
- **Querying**: Use `notion.dataSources.query(dataSourceId) { }` with optional filters and sorting
- **Filters**: Combine conditions with `and()` and `or()` for powerful queries
- **Sorting**: Use `sortBy()` with `SortDirection.ASCENDING` or `DESCENDING`
- **Pagination**: Handled automatically - you always get all matching pages
- **Property Access**: Use type-safe pattern matching with `when` to access different property types